# Productionisation of ML Models (Model Serialization):


  ### Loading the data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data = pd.read_csv("data.csv")
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [7]:
print(data.shape)

(569, 33)


In [9]:
data.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

## Setting the target and input features:

In [12]:
X = data[['radius_mean', 'texture_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean']]
y = data['diagnosis']

## Test Train data spliting:

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

## Data preprocessing on the Training data first:

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_rescaled = pd.DataFrame(scaler.fit_transform(X_train), 
                                    columns = X_train.columns, 
                                    index = X_train.index)

X_train_rescaled.head()

,radius_mean,texture_mean,smoothness_mean,compactness_mean,concavity_mean
31,-0.651891,-0.156803,1.068831,0.907648,0.438137
77,1.120901,-0.760780,0.746028,2.122563,1.042224
40,-0.195133,0.525337,-1.079276,-0.852822,-0.737628
519,-0.392110,-0.630511,1.186214,0.138202,-0.637811
215,-0.075234,-0.576034,0.459907,0.909577,0.142705


In [15]:
scaler.var_, scaler.scale_

(array([1.22706477e+01, 1.78253726e+01, 1.85793496e-04, 2.68899024e-03,
        5.95077525e-03]),
 array([3.50294843, 4.22201049, 0.01363061, 0.05185547, 0.07714127]))

## Preprocessing the Test data next:

In [16]:
X_test_rescaled = pd.DataFrame(scaler.transform(X_test), 
                                   columns = X_test.columns, 
                                   index = X_test.index)
X_test_rescaled.head()

,radius_mean,texture_mean,smoothness_mean,compactness_mean,concavity_mean
123,0.107469,-2.006632,1.010139,0.103490,0.005425
30,1.286476,1.361431,0.738691,1.623098,1.865389
370,0.635596,0.930357,0.079880,0.871008,1.206857
262,0.903941,0.655607,-0.465217,0.439038,0.116260
89,0.147436,-0.976317,1.237569,0.566315,0.151131


# (1.) Training the KNN classifier algo:

In [17]:
from sklearn.neighbors import KNeighborsClassifier
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(X_train_rescaled, y_train)

KNeighborsClassifier()

## KNN prediction:

In [18]:
y_test_pred = knn_classifier.predict(X_test_rescaled)

## KNN accuracy check:

In [19]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_test_pred)


0.956140350877193

# (2). Training the Naive bayes algo:

In [20]:
from sklearn.naive_bayes import GaussianNB
nb_classifier = GaussianNB()
nb_classifier.fit(X_train_rescaled, y_train)

GaussianNB()

## Naive bayes prediction :

In [21]:
y_test_pred = nb_classifier.predict(X_test_rescaled)

## Naive bayes accuracy check:

In [22]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_test_pred)

0.9736842105263158

# (3). Training the Logistic regression algo:

In [23]:
from sklearn.linear_model import LogisticRegression
lr_classifier = LogisticRegression()
lr_classifier.fit(X_train_rescaled, y_train)

LogisticRegression()

## Logistic regression prediction:

In [24]:
y_test_pred = lr_classifier.predict(X_test_rescaled)

## Logistic regression accuracy check :

In [25]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_test_pred)

0.9473684210526315

# (4). Training Deciosion tree algo:

In [26]:
from sklearn.tree import DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(max_depth = 4)
dt_classifier.fit(X_train_rescaled, y_train)

DecisionTreeClassifier(max_depth=4)

## Decision tree prediction:

In [27]:
y_test_pred = dt_classifier.predict(X_test_rescaled)

## Decision tree accuracy check:


In [28]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_test_pred)

0.9649122807017544

# SERIALIZATION : (dumping the model into drive)

In [29]:
from pickle import dump

dump(scaler, open('models/standard_scaler.pkl', 'wb'))
dump(knn_classifier, open('models/knn_model.pkl', 'wb'))
dump(lr_classifier, open('models/lr_model.pkl', 'wb'))
dump(nb_classifier, open('models/nb_model.pkl', 'wb'))
dump(dt_classifier, open('models/dt_model.pkl', 'wb'))